In [1]:
# Transformers installation
! pip install transformers datasets
! pip install seqeval
! pip install evaluate
! pip install wandb

import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=bc1045a5e51ed2c7d60ca942945c678e08dc1a99c73f3dc2577d3c5f43fb903d
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abf

In [2]:
# model_name = "haisongzhang/roberta-tiny-cased" # https://github.com/haisongzhang/roberta-tiny-cased
# model_name = "distilbert-base-uncased" # https://github.com/huggingface/transformers/tree/main/examples/research_projects/distillation
model_name = "distilroberta-base" # https://huggingface.co/distilroberta-base
# model_name = "roberta-base" # https://huggingface.co/roberta-base

# Preprocess Data


## Load Tokenizer and Dataset

Inspec Dataset has 3 datasets: train, test, and validation

### Data Types

*   train: [*sample*]
*   *sample*: {
  document: [string],
  doc_bio_tags: [int]
  }

In [3]:
from datasets import load_dataset

inspec = load_dataset("midas/inspec")

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [4]:
example = inspec["train"][0]

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

## Preprocess

We need to


*   Convert words to tokens
*   Realign the tokens and labels, since a single word may correspond to multiple tokens.
*   Ignore the special tokens [CLS] and [SEP] in the loss function.
*   Only label the first token of a given word, and assign -100 to the others.
*   Convert doc_bio_tags to integers for classificatioon



In [6]:
label_list = np.unique(example["doc_bio_tags"])

id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

print('Mapping doc_bio_tag to integer:\n\n',label2id)
print('\nMapping integer to doc_bio_tag:\n\n',id2label)

Mapping doc_bio_tag to integer:

 {'B': 0, 'I': 1, 'O': 2}

Mapping integer to doc_bio_tag:

 {0: 'B', 1: 'I', 2: 'O'}


In [7]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["document"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"doc_bio_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label2id[label[word_idx]]) # Convert BIO to integers for classification
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [8]:
# This is our tokenized dataset (a data_dict which we will convert to a TF Dataset)
tokenized_inspec = inspec.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

## Utils for Training

In [9]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [10]:
import evaluate

seqeval = evaluate.load("seqeval")

In [11]:
labels = example[f"doc_bio_tags"]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    log_results = {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }
    wandb.log(log_results)

    return log_results

## Experiment Tracking with W&B

In [12]:
import wandb

wandb.login()
wandb.init(entity="ac215-ppp", project="ppp-keyword-extraction", name=f"{model_name}-21oct")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: benray (ac215-ppp). Use `wandb login --relogin` to force relogin


## These variables will be used to define a custom loss with weighted cross entropy

In [13]:
from collections import Counter

#counting how many beginning keywords, middle keywords, and non-keywords there are
count_0s = 0
count_1s = 0
count_2s = 0

for listt in tokenized_inspec["train"]["labels"]:
    count_dict = Counter(listt)
    count_0s += count_dict[0]
    count_1s += count_dict[1]
    count_2s += count_dict[2]

#getting weights for weighted cross_entropy
max_ = max(count_0s,count_1s,count_2s)
weights = [max_/count_0s, max_/count_1s, max_/count_2s]

## Train with multi-GPU
Reference: https://saturncloud.io/docs/examples/python/tensorflow/qs-multi-gpu-tensorflow/

In [14]:
import tensorflow as tf
import keras
import time
from transformers import TFAutoModelForTokenClassification
from transformers.keras_callbacks import KerasMetricCallback
from transformers import create_optimizer

def train_multigpu(n_epochs, base_lr, batchsize):
    num_train_steps = (len(tokenized_inspec["train"]) // batchsize) * n_epochs
    num_labels=3

    # Set up for multi-GPU training
    strategy = tf.distribute.MirroredStrategy()
    print("Number of devices: %d" % strategy.num_replicas_in_sync)

    with strategy.scope():
        model = TFAutoModelForTokenClassification.from_pretrained(
                  model_name, num_labels=num_labels, id2label=id2label, label2id=label2id)

        # We define our own optimizer (and lr_schedule which we do not use)
        optimizer, lr_schedule = create_optimizer(
                                                  init_lr=2e-5,
                                                  num_train_steps=num_train_steps,
                                                  weight_decay_rate=0.01,
                                                  num_warmup_steps=0,
                                                 )

        # Compute custom loss (CrossEntropyLoss with weights)
        def loss_fn(y_true, y_pred):
          loss = tf.nn.weighted_cross_entropy_with_logits(
              labels=tf.one_hot(y_true, depth=num_labels),
              logits=y_pred,
              pos_weight=tf.constant(weights)
          )
          loss = tf.reduce_mean(loss)

          return loss

        # The model is ready for training
        model.compile(loss=loss_fn, optimizer=optimizer, metrics=["accuracy"])

    # Load in our data as TF Datasets (with data_collator applied)
    train_ds = model.prepare_tf_dataset(
                                        tokenized_inspec["train"],
                                        shuffle=True,
                                        batch_size=batchsize,
                                        collate_fn=data_collator,
                                       ).prefetch(2).cache().shuffle(1000)

    valid_ds = model.prepare_tf_dataset(
                                        tokenized_inspec["validation"],
                                        shuffle=False,
                                        batch_size=batchsize,
                                        collate_fn=data_collator,
                                       ).prefetch(2)

    # Run training
    start = time.time()

    model.fit(x=train_ds, validation_data=valid_ds, epochs=n_epochs, callbacks=[wandb.keras.WandbCallback()])

    end = time.time() - start
    print("model training time", end)

    # Save the trained model as a W&B artifact
    with wandb.init() as run:
        # Create a W&B artifact to save the model
        trained_model_artifact = wandb.Artifact("trained_model", type="model")
        # Save the model to a specified directory (adjust the path)
        model.save_pretrained("model_directory")
        # Add the saved model to the artifact
        trained_model_artifact.add_dir("model_directory")
        # Log the artifact
        run.log_artifact(trained_model_artifact)

In [15]:
# Call the training function with specified parameters
model_params = {
    "n_epochs": 5,
    "base_lr": 2e-5,
    "batchsize": 16
}

tester_plain = train_multigpu(**model_params)

Number of devices: 1


All PyTorch model weights were used when initializing TFRobertaForTokenClassification.

Some weights or buffers of the TF 2.0 model TFRobertaForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/5
 6/62 [=>............................] - ETA: 37s - loss: 0.7405 - accuracy: 0.0703

62/62 [==============================] - ETA: 0s - loss: 0.3989 - accuracy: 0.3117

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20231022_004220-o0up67jr/files/model-best)... Done. 4.5s


...

62/62 [==============================] - 65s 1s/step - loss: 0.1274 - accuracy: 0.3527 - val_loss: 0.1408 - val_accuracy: 0.3608
Epoch 5/5
62/62 [==============================] - ETA: 0s - loss: 0.1218 - accuracy: 0.3548

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20231022_004220-o0up67jr/files/model-best)... Done. 3.9s


62/62 [==============================] - 65s 1s/step - loss: 0.1218 - accuracy: 0.3548 - val_loss: 0.1392 - val_accuracy: 0.3587
model training time 347.77565360069275


accuracy,▁▆▇██
epoch,▁▃▅▆█
loss,█▂▁▁▁
val_accuracy,▁▆▇█▇
val_loss,█▂▁▁▁
accuracy,0.35482
best_epoch,4
best_val_loss,0.13916
epoch,4
loss,0.12175
val_accuracy,0.35867


wandb: Adding directory to artifact (./model_directory)... Done. 0.9s
